In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from kafka import KafkaProducer
import time

# Create a producer object
# Ip can be different in your case
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])
def Convert(a):
    it = iter(a)
    res_dct = dict(zip(it, it))
    return res_dct
df1=pd.DataFrame(columns=['İlan No:', 'İlan Tarihi:', 'Marka:', 'Seri:', 'Model:', 'Yıl:',
       'Yakıt Tipi:', 'Vites Tipi:', 'Motor Hacmi:', 'Motor Gücü:',
       'Kilometre:', 'Boya-değişen:', 'Takasa Uygun:', 'Kimden:'])
df2=pd.DataFrame(columns=['İlan No:', 'İlan Tarihi:', 'Marka:', 'Seri:', 'Model:', 'Yıl:',
       'Yakıt Tipi:', 'Vites Tipi:', 'Motor Hacmi:', 'Motor Gücü:',
       'Kilometre:', 'Boya-değişen:', 'Takasa Uygun:', 'Kimden:'])
while 1==1:
    for page in [1]:
          
      
          url="https://www.arabam.com/ikinci-el/otomobil?page="+str(page)

          r=requests.get(url)
          soup=BeautifulSoup(r.content,'html.parser')
          table=soup.find("table",attrs={"class":"table listing-table w100 border-grey2"})
          cars=table.find_all("a",attrs={"class":"listing-text-new word-break val-middle color-black2018"})
          for car in cars:
            car_link=[]
            car_Link=car.get("href")
            car_Link="https://www.arabam.com/"+car_Link
            car_link.append(car_Link)
            #car_r=requests.get(car_Link)
            soup = BeautifulSoup(requests.get(car_Link).content, 'html.parser')
            table1=soup.find_all("span",attrs={"class":"bli-particle"})
            att=[]

            for i in table1:
                att.append(i.text)
            price=soup.find("div",attrs={"class":"mb8"}).text
                
               
            data=Convert(att)
            df=pd.DataFrame.from_dict(data, orient="index").T
            df["car_link:"]=car_link
            df["price"]=price
            
            
            df1=pd.concat([df1,df],ignore_index=True,join="outer")
    df1.drop_duplicates(subset=['İlan No:'], keep='last',inplace=True)
    
    df1=df1.loc[:,['İlan No:', 'İlan Tarihi:', 'Marka:', 'Seri:', 'Model:', 'Yıl:','Yakıt Tipi:', 'Vites Tipi:', 'Motor Hacmi:', 'Motor Gücü:',"Kilometre:",'car_link:',"price"]]
    df3=df1[~df1['İlan No:'].isin(list(df2['İlan No:']))]
    df2=pd.concat([df2,df1],ignore_index=True,join="outer")
    df2=df2.loc[:,['İlan No:', 'İlan Tarihi:', 'Marka:', 'Seri:', 'Model:', 'Yıl:','Yakıt Tipi:', 'Vites Tipi:', 'Motor Hacmi:', 'Motor Gücü:',"Kilometre:",'car_link:',"price"]]
    df1.drop_duplicates(subset=['İlan No:'], keep='last',inplace=True)
    df1.to_csv('arabam.csv')
    df2.drop_duplicates(subset=['İlan No:'], keep='last',inplace=True)
    df3.drop_duplicates(subset=['İlan No:'], keep='last',inplace=True)

    # Read csv file and quick overview

    df3=df3.loc[:,['İlan No:', 'İlan Tarihi:', 'Marka:', 'Seri:', 'Model:', 'Yıl:','Yakıt Tipi:', 'Vites Tipi:', 'Motor Hacmi:', 'Motor Gücü:',"Kilometre:",'car_link:',"price"]]
    


    # DRop null
    df3=df3.dropna()
    if len(df3['İlan No:'])>0:
        import numpy as np
        for col in df3.columns:
            df3[col]=np.where(1==1,df3[col]+",",df3[col]+",")
        

        # Put columns (including key column) to produce in a single columns named value
        
        x = df3.to_string(header=False,
                          index=False,
                          index_names=False)
        

        df3['value'] = [x.split("\n")[i] for i in range(0,(len(x.split("\n"))))]



        # Iterate through dataframe and produce it
        for index, row in df3.iterrows():
            time.sleep(0.1)
            producer.send('topic1', key=str(row[0]).encode(), value=row[-1].encode())
            print('topic1', "key=",str(row[0]).encode(), "value=",row[-1].encode())

        # Observe results from three console consumer under group1
   

topic1 key= b' 16270245   ,' value= b'  16270245   ,   25 Aral\xc4\xb1k 2020   ,   Volkswagen   ,              Passat   ,    1.6 TDi BlueMotion Comfortline   ,   2014   ,          Dizel   ,        Otomatik   ,   1401 - 1600 cm3   ,   101 - 125 HP   ,   243.000 km   ,  https://www.arabam.com//ilan/galeriden-satilik-volkswagen-passat-1-6-tdi-bluemotion-comfortline/2014-model-otomotik-vites-dizel-vw-passat-bluemotion-comfortline-paket/16270245,    187.900 TL  ,'
topic1 key= b' 16270223   ,' value= b'  16270223   ,   25 Aral\xc4\xb1k 2020   ,         Seat   ,                Leon   ,                     1.6 TDI Style   ,   2018   ,          Dizel   ,        Otomatik   ,   1401 - 1600 cm3   ,   101 - 125 HP   ,   132.000 km   ,                                   https://www.arabam.com//ilan/galeriden-satilik-seat-leon-1-6-tdi-style/otomatik-vites-2018-hayalet-ekran-led-far-buyuk-ekran-car-play/16270223,    245.000 TL  ,'
topic1 key= b' 16270149   ,' value= b'  16270149   ,   25 Aral\xc4\xb1k 

topic1 key= b' 16267136   ,' value= b'  16267136   ,   25 Aral\xc4\xb1k 2020   ,      Peugeot   ,                 508   ,                    1.6 HDi Active   ,   2012   ,          Dizel   ,   Yar\xc4\xb1 Otomatik   ,                 -   ,              -   ,   190.000 km   ,                                             https://www.arabam.com//ilan/sahibinden-satilik-peugeot-508-1-6-hdi-active/sahibinden-peugeot-508-1-6-hdi-active-2012-model/16267136,    140.000 TL  ,'
